In [1]:
!pip install -q flask pyngrok pyspark flask-cors


In [4]:
from flask import Flask, request, jsonify
from pyspark.sql import SparkSession
from pyspark.ml import PipelineModel
from pyspark.ml.regression import RandomForestRegressionModel
from pyngrok import ngrok
from flask_cors import CORS
from google.colab import drive
drive.mount('/drive')

# Start Spark
spark = SparkSession.builder.appName("TRAV_SP_Predictor").getOrCreate()

# Load models
pipeline_model = PipelineModel.load("/drive/My Drive/Cal State Fullerton/ADBMS Final Project/pipeline_model")
rf_model = RandomForestRegressionModel.load("/drive/My Drive/Cal State Fullerton/ADBMS Final Project/rf_model")
app = Flask(__name__)
CORS(app)
@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = request.json
        print("Incoming data:", data)

        # Create Spark DataFrame
        input_df = spark.createDataFrame([data])
        print("Spark DataFrame:")
        input_df.show()

        # Transform
        features_df = pipeline_model.transform(input_df)
        prediction_result = rf_model.transform(features_df)

        # Extract prediction
        prediction = prediction_result.select("prediction").collect()[0][0]

        return jsonify({"predicted_TRAV_SP": round(prediction, 2)})

    except Exception as e:
        import traceback
        traceback.print_exc()
        return jsonify({"error": str(e)}), 500


Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [5]:
!ngrok config add-authtoken 2wplMlnjynx2gKfDdx2innf6uyc_2GdqWCFcBXk2pucLd7Kdo

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
# Run Flask with ngrok
public_url = ngrok.connect(5000)
print("Ngrok URL:", public_url)
# CORS(app)
app.run(port=5000)

Ngrok URL: NgrokTunnel: "https://7bae-34-42-149-245.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [09/May/2025 23:19:13] "OPTIONS /predict HTTP/1.1" 200 -


Incoming data: {'AGE': 35, 'NUMOCCS': 2, 'MOD_YEAR': 2017, 'HOUR': 10, 'PSU': 101, 'WEIGHT': 1500, 'NUM_VEH': 1, 'POP2018': 50000, 'PREV': 0, 'PREV_ACC': 0, 'PREV_SUS': 0, 'PREV_DWI': 0, 'PREV_SPD': 0, 'GENDER': '2', 'PER_TYP': '1', 'INJ_SEV': '3', 'DRINKING': '0', 'DRUGS': '0', 'MAKE': '10', 'MODEL': '25', 'HIT_RUN': '0', 'BODY_TYP': '4', 'DEFORMED': '6', 'SPEC_USE': '0', 'SPEEDREL': '1', 'DR_SF1': '1', 'HARM_EV': '4', 'WEATHER': '0', 'STRATUM': '2', 'REGION': '3', 'PJ': '2', 'MAX_SEV': '3', 'MARITAL': 'Single', 'RACE': 'Asian', 'DR_DRINK': '0', 'CDL_STAT': '0'}
Spark DataFrame:
+---+--------+--------+--------+--------+-----+--------+------+------+-------+-------+----+-------+----+-------+-------+-----+--------+-------+-------+-------+---+-------+----+--------+--------+--------+--------+---+-----+------+--------+--------+-------+-------+------+
|AGE|BODY_TYP|CDL_STAT|DEFORMED|DRINKING|DRUGS|DR_DRINK|DR_SF1|GENDER|HARM_EV|HIT_RUN|HOUR|INJ_SEV|MAKE|MARITAL|MAX_SEV|MODEL|MOD_YEAR|NUMOCCS

INFO:werkzeug:127.0.0.1 - - [09/May/2025 23:19:19] "POST /predict HTTP/1.1" 200 -
